In [1]:
from pathlib import Path

import dotenv
import pytorch_lightning as pl
import torch
from matplotlib import pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint, RichProgressBar

In [2]:
from inz.data.data_module import XBDDataModule
from inz.data.event import Event, Tier3, Tier1
from inz.models.baseline_module import BaselineModule
from inz.util import get_loc_cls_weights, get_wandb_logger, show_masks_comparison
from inz.xview2_strong_baseline.legacy.losses import ComboLoss
from inz.xview2_strong_baseline.legacy.zoo.models import Res34_Unet_Double

In [3]:
import hiddenlayer as hl
import torchviz as tv
import torchview as tv2

In [4]:
model = BaselineModule(
    model=Res34_Unet_Double(pretrained=True),
    loss=ComboLoss(weights={"dice": 1, "focal": 1}),
    optimizer_factory=lambda params: torch.optim.AdamW(params, lr=0.0002, weight_decay=1e-6),
    scheduler_factory=lambda optimizer: torch.optim.lr_scheduler.MultiStepLR(
        optimizer=optimizer,
        gamma=0.5,
        milestones=[5, 11, 17, 23, 29, 33, 47, 50, 60, 70, 90, 110, 130, 150, 170, 180, 190],
    ),
)

using weights from ResNet34_Weights.IMAGENET1K_V1


In [27]:
# hl.build_graph(model, torch.zeros([1, 6, 512, 512]))

# y = model.forward(torch.zeros([1, 6, 512, 512]))
# tv.make_dot(y.mean(), dict(model.named_parameters()))

model_graph = tv2.draw_graph(
    model=model,
    input_size=(1, 6, 512, 512),
    depth=5,
    roll=True,
    # show_shapes=False,
    expand_nested=True,
    graph_dir="LR",
)
model_graph.visual_graph